# Extracts & cleans data from Spotify using spotipy API

In [1]:
import os
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID = os.environ.get('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.environ.get('SPOTIFY_CLIENT_SECRET')

# spotify auth
auth_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
from dg_tools import SpotifyTools as st

# fetch classical essentials playlist & all its track audio features
ce_id = '37i9dQZF1DWWEJlAGA9gs0' # classical essentials playlist id
ce_df = st.get_spotify_playlist(ce_id,sp)
ce_af = st.get_spotify_track_audio_features(ce_df.id,sp)
ce_df = ce_df.join(ce_af)

In [10]:
ce_df

,name,artist_name,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,"The Well-Tempered Clavier: Book 1, BWV 846-869...","[Johann Sebastian Bach, Lang Lang]",4SFBV7SRNG2e2kyL1F6kjU,0.2210,0.1260,0,-25.427,1,0.0447,0.989,0.89700,0.1020,0.2160,133.630
1,The Lark Ascending,"[Ralph Vaughan Williams, Tasmin Little, BBC Ph...",2kAgCRZPG3YQR2VMqRvLmb,0.0811,0.0122,4,-32.654,0,0.0511,0.902,0.30800,0.0648,0.0384,74.554
2,Handel / Orch. Hale: Keyboard Suite in D Minor...,"[George Frideric Handel, Alexander Briger, Aca...",1upQiytDIEZfl9ItruoXuC,0.2040,0.0331,2,-24.044,0,0.0487,0.927,0.82500,0.0954,0.0547,116.200
3,"Bagatelle No. 25 in A Minor, ""Für Elise"", WoO 59","[Ludwig van Beethoven, Lang Lang]",0H2VhGUC3P3hvwz8rdGIpC,0.2300,0.0487,5,-27.675,1,0.0412,0.986,0.94500,0.0679,0.1010,134.839
4,"Rhapsody On A Theme Of Paganini, Op.43: Variat...","[Sergei Rachmaninoff, Daniil Trifonov, Philade...",419qlOGNXoyICx4jO0hqaj,0.2050,0.0525,1,-23.841,1,0.0327,0.980,0.90600,0.1030,0.0511,88.316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,The Firebird (L'oiseau De Feu) - Suite (1919):...,"[Igor Stravinsky, Orchestre de l'Opéra Bastill...",0mTUTkFQ2MxmUHFL8EdfeR,0.1230,0.2250,4,-16.798,1,0.0351,0.730,0.50900,0.0658,0.0550,78.132
168,"Holberg Suite, Op. 40: 1. Präludium (Allegro v...","[Edvard Grieg, Gothenburg Symphony Orchestra, ...",07xafomqQcYmFJbr4jpfHa,0.2370,0.0606,7,-22.569,1,0.0374,0.910,0.93900,0.1310,0.2560,136.284
169,"Die Forelle, Op. 32, D. 550","[Franz Schubert, Dietrich Fischer-Dieskau, Ger...",7lCqzDWT6GPECGp0TzMnR7,0.5140,0.0772,11,-21.076,1,0.0352,0.981,0.00306,0.0859,0.5110,87.514
170,"Cello Concerto No. 1 in C Major, Hob.VIIb:1: I...","[Franz Joseph Haydn, Steven Isserlis, Sir Roge...",0Gh45IbIKOG9IucFfrZqLT,0.3750,0.1210,0,-23.440,1,0.0406,0.984,0.87100,0.0984,0.4530,147.398


In [ ]:
ce_df.to_csv('classical_essentials')

In [ ]:
# gets a feature/obj from a 2d list
# lst: list, obj: str/int
get_obj = lambda lst, obj : [lst[i][obj] for i in range(len(lst))] 

# refines a list of playlist items
# assumed Spotify obj is authenticated
def get_spotify_playlist(_id: int, sp: spotipy.Spotify) -> pd.DataFrame:
    _playlist = sp.playlist_tracks(_id) # fetch playlist
    track_count = _playlist['total']
    
    # 100 item limit per request
    while track_count != len(_playlist['items']):
        batch = sp.playlist_tracks(_id, offset = len(_playlist['items']))
        _playlist['items'].extend(batch['items'])
    
    items = _playlist['items']
    tracks = get_obj(items,'track')
    artists = get_obj(tracks,'artists')
    
    artist_names = []
    for ix in range(track_count):
        z = []
        for jx in range(len(artists[ix])):
            z.append(artists[ix][jx]['name'])
        artist_names.append(z)
    
    return pd.DataFrame({
        'name': get_obj(tracks,'name'),
        'artist_name': artist_names,
        'id': get_obj(tracks,'id')
    })

# returns audio features given a list of track ids
def get_spotify_track_audio_features(_ids: list, sp: spotipy.Spotify) -> pd.DataFrame:
    audio_features = []
    total = len(_ids)
    
    # 100 item limit per request
    for i in range(total//100):
        audio_features.extend(sp.audio_features(_ids[i*100:i*100+100]))
    audio_features.extend(sp.audio_features(_ids[len(audio_features):total]))
    
    # convert to pandas dataframe
    return pd.DataFrame({
        'danceability': get_obj(audio_features,'danceability'),
        'energy': get_obj(audio_features,'energy'),
        'key': get_obj(audio_features,'key'),
        'loudness': get_obj(audio_features,'loudness'),
        'mode': get_obj(audio_features,'mode'),
        'speechiness': get_obj(audio_features,'speechiness'),
        'acousticness': get_obj(audio_features,'acousticness'),
        'instrumentalness': get_obj(audio_features,'instrumentalness'),
        'liveness': get_obj(audio_features,'liveness'),
        'valence': get_obj(audio_features,'valence'),
        'tempo': get_obj(audio_features,'tempo')
    })